Notes:

Model - 
lstm model (2 layers) using trigrams without word2vec, ran with 25 epochs

Data -
all taylor swift lyrics split in trigram sequences

Results -
pretty poor. approx 5% accuracy and nonsensical sentences. Accuracy did not seem to grow

In [8]:
import pandas as pd
import numpy as np
from nltk import word_tokenize, ngrams
import contractions
from keras.preprocessing.text import Tokenizer
import keras.utils.np_utils as ku
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

In [18]:
NGRAM_SIZE = 3

In [9]:
def csv_to_lst(file):
    """
    Grabs the lyric section of the given csv and convert it to a string
    Parameters:
        file (CSV): file path where csv is located
    Returns:
        A list string representing all of the text section in the csv
    """
    lyrics_df = pd.read_csv(file) 
#     lyrics_df = lyrics_df

    lyrics_list = list(lyrics_df['lyrics'])


    return lyrics_list

In [10]:
def pre_process_all_lyrics(lyric_list):
    processed = []
    for lyrics in lyric_list:
#         print(lyrics)
        processed.append(pre_process_text(lyrics))
    return processed

In [11]:
def pre_process_line(line):
    EMBED = 'embed'
    word_list = word_tokenize(line)
    
   
    #index = word_list.index('Lyrics')
    #word_list = word_list[index + 1:]
    
    no_brackets_list = []
    is_inside = False
    punctuation_list = '''!()-[];:'"\,<>./?@#$%^&*_~”“’‘--...``'''''
    
    for word in word_list:
        if word == '[':
            is_inside = True
        if word == ']':
            is_inside = False
        else:
            if not is_inside:
                if word not in punctuation_list:
                    no_brackets_list.append(word.lower())

    if len(no_brackets_list) > 0:
        end_word = no_brackets_list[len(no_brackets_list)-1]   
        if EMBED in end_word:
#             print(end_word)
            embed_location = end_word.index('embed') 
            # all lyric genius data comes with word 'Embed' at the end
            substr = end_word[:embed_location]
            if len(substr) > 0:
                no_brackets_list[len(no_brackets_list)-1] = substr
        
        end_word = no_brackets_list[len(no_brackets_list)-1]  
        new_end_word = ''
        # also sometimes has a number before embed in last word
        for c in end_word:
            if c.isdigit():
                break
            else:
                new_end_word = new_end_word + c
        if len(end_word) > 0:
            no_brackets_list[len(no_brackets_list)-1] = new_end_word
            
    ret = []
    for word in no_brackets_list:
        if len(word) < 0:
            break
        #idk why this is like this 
        elif word == "''":
            break
        else:
            ret.append(word)
    return ret

In [12]:
def pre_process_text(lyrics):
    """
    Preprocesses the text to add start and end tokens to each sentence
    Parameters:
        lst (list): list of lyrics 
    Returns:
        List of List of words with start and end tokens
    """
    
    lyrics = contractions.fix(lyrics)
#     print(lyrics)
    line_list = lyrics.split("\n")
    line_list = line_list[1:]
    
    ret = []
    for line in line_list:
        if not len(line) == 0:
            to_be_appended = pre_process_line(line)
            if len(to_be_appended) > 0:
                ret.append(to_be_appended)
    return ret

In [13]:
lst = csv_to_lst('taylorswift.csv')
# print(lst)
all_songs = pre_process_all_lyrics(lst)

In [14]:
for song in all_songs:
    for line in song:
        print(line)

['i', 'walked', 'through', 'the', 'door', 'with', 'you', 'the', 'air', 'was', 'cold']
['but', 'something', "'bout", 'it', 'felt', 'like', 'home', 'somehow']
['and', 'i', 'left', 'my', 'scarf', 'there', 'at', 'your', 'sister', "'s", 'house']
['and', 'you', 'have', 'still', 'got', 'it', 'in', 'your', 'drawer', 'even', 'now']
['oh', 'your', 'sweet', 'disposition', 'and', 'my', 'wide-eyed', 'gaze']
['we', 'are', 'singin', 'in', 'the', 'car', 'getting', 'lost', 'upstate']
['autumn', 'leaves', 'fallin', 'down', 'like', 'pieces', 'into', 'place']
['and', 'i', 'can', 'picture', 'it', 'after', 'all', 'these', 'days']
['and', 'i', 'know', 'it', 'is', 'long', 'gone', 'and']
['that', 'magic', "'s", 'not', 'here', 'no', 'more']
['and', 'i', 'might', 'be', 'okay', 'but', 'i', 'am', 'not', 'fine', 'at', 'all']
['oh', 'oh', 'oh']
["'causе", 'there', 'we', 'arе', 'again', 'on', 'that', 'little', 'town', 'street']
['you', 'almost', 'ran', 'the', 'red', 'because', 'you', 'were', 'lookin', 'over', 'at', '

['family', 'that', 'i', 'chose', 'now', 'that', 'i', 'see', 'your', 'brother', 'as', 'my', 'brother']
['is', 'it', 'enough']
['but', 'there', 'is', 'robbers', 'to', 'the', 'east', 'clowns', 'to', 'the', 'west']
['i', 'would', 'give', 'you', 'my', 'sunshine', 'give', 'you', 'my', 'best']
['but', 'the', 'rain', 'is', 'always', 'going', 'to', 'come', 'if', 'you', 'are', 'standin', 'with', 'me']
['but', 'i', 'am', 'a', 'fire', 'and', 'i', 'will', 'keep', 'your', 'brittle', 'heart', 'warm']
['if', 'your', 'cascade', 'ocean', 'wave', 'blues', 'come']
['all', 'these', 'people', 'think', 'love', "'s", 'for', 'show']
['but', 'i', 'would', 'die', 'for', 'you', 'in', 'secret']
['the', 'devil', "'s", 'in', 'the', 'details', 'but', 'you', 'got', 'a', 'friend', 'in', 'me']
['would', 'it', 'be', 'enough', 'if', 'i', 'could', 'never', 'give', 'you', 'peace']
['would', 'it', 'be', 'enough', 'if', 'i', 'could', 'never', 'give', 'you', 'peace']
['would', 'it', 'be', 'enough', 'if', 'i', 'could', 'never',

['it', 'is', 'all', 'just', 'a', 'blur']
['i', 'forgot', 'that', 'you', 'existed']
['and', 'i', 'thought', 'that', 'it', 'would', 'kill', 'me', 'but', 'it', 'did', 'not']
['and', 'it', 'was', 'so', 'nice']
['so', 'peaceful', 'and', 'quiet']
['i', 'forgot', 'that', 'you', 'existed']
['i', 'did', 'i', 'did', 'i', 'did']
['it', 'is', 'not', 'hate', 'it', 'is', 'just', 'indifference']
['it', 'is', 'not', 'love', 'it', 'is', 'not', 'hate', 'it', 'is', 'just', 'indifference']
['so', 'yeah']
['and', 'the', 'tennis', 'court', 'was', 'covered', 'up']
['with', 'some', 'tent-like', 'thing']
['and', 'you', 'asked', 'me', 'to', 'dance']
['but', 'i', 'said', 'dancin', 'is', 'a', 'dangerous', 'game']
['oh', 'i', 'thought']
['this', 'is', 'going', 'to', 'be', 'one', 'of', 'those', 'things']
['now', 'i', 'know']
['i', 'am', 'never', 'going', 'to', 'love', 'again']
['i', 'have', 'got', 'some', 'tricks', 'up', 'my', 'sleeve']
['takes', 'one', 'to', 'know', 'one']
['you', 'are', 'a', 'cowboy', 'like', 'me

['and', 'now', 'i', 'see', 'now', 'i', 'see']
['now', 'i', 'see-e-e-e-e']
['he', 'was', 'long', 'gone', 'when', 'he', 'met', 'me']
['and', 'i', 'realize', 'the', 'joke', 'is', 'on', 'me', 'yeah']
['i', 'knew', 'you', 'were', 'trouble', 'when', 'you', 'walked', 'in', 'oh']
['so', 'shame', 'on', 'me', 'now']
['i', 'will', 'flew', 'me', 'to', 'places', 'i', 'would', 'never', 'been']
["'til", 'you', 'put', 'me', 'down', 'oh']
['i', 'knew', 'you', 'were', 'trouble', 'when', 'you', 'walked', 'in']
['so', 'shame', 'on', 'me', 'now']
['flew', 'me', 'to', 'places', 'i', 'would', 'never', 'been', 'yeah']
['now', 'i', 'am', 'lying', 'on', 'the', 'cold', 'hard', 'ground']
['oh', 'oh-oh']
['trouble', 'trouble', 'trouble', 'yeah', 'trouble']
['oh', 'oh-oh']
['trouble', 'trouble', 'trouble']
['and', 'the', 'saddest', 'fear']
['comes', 'creeping', 'in']
['that', 'you', 'never', 'loved', 'me']
['or', 'her', 'or', 'anyone', 'or', 'anything', 'yeah']
['i', 'knew', 'you', 'were', 'trouble', 'when', 'you',

['you', 'have', 'knocked', 'me', 'off', 'my', 'feet', 'again']
['got', 'me', 'feeling', 'like', 'i', 'am', 'nothing']
['you', 'with', 'your', 'voice', 'like', 'nails', 'on', 'a', 'chalkboard']
['calling', 'me', 'out', 'when', 'i', 'am', 'wounded']
['you', 'picking', 'on', 'the', 'weaker', 'man']
['well', 'you', 'can', 'take', 'me', 'down']
['with', 'just', 'one', 'single', 'blow']
['but', 'you', 'do', 'not', 'know', 'what', 'you', 'do', 'not', 'know']
['someday', 'i', 'will', 'be', 'living', 'in', 'a', 'big', 'old', 'city']
['and', 'all', 'you', 'are', 'ever', 'going', 'to', 'be', 'is', 'mean']
['someday', 'i', 'will', 'be', 'big', 'enough', 'so', 'you', 'can', 'not', 'hit', 'me']
['and', 'all', 'you', 'are', 'ever', 'going', 'to', 'be', 'is', 'mean']
['why', 'you', 'got', 'to', 'be', 'so', 'mean']
['you', 'with', 'your', 'switching', 'sides']
['and', 'your', 'wildfire', 'lies', 'and', 'your', 'humiliation']
['you', 'have', 'pointed', 'out', 'my', 'flaws', 'again']
['as', 'if', 'i', 'd

...

['i', 'am', 'runnin', 'right', 'out', 'to', 'my', 'car']
['and', 'i', 'pull', 'out', 'the', 'drive']
['it', 'is', 'only', 'been', 'ten', 'minutes']
['but', 'i', 'have', 'to', 'see', 'you', 'one', 'more', 'time']
['boy', 'you', 'got', 'me', 'like', 'a', 'shot', 'to', 'the', 'heart']
['got', 'me', 'shakin', 'so', 'bad', 'spilled', 'my', 'coffee', 'in', 'the', 'car']
['caught', 'a', 'red', 'light', 'i', 'ran', 'right', 'through']
['got', 'to', 'call', 'my', 'friends', 'tell', 'them', 'all', 'about', 'you']
['always', 'looking', 'like', 'one', 'of', 'those', 'nights']
['when', 'you', 'are', 'thinkin', "'bout", 'kissing', "'til", 'the', 'morning', 'light']
['you', 'got', 'me', 'you', 'do']
['there', 'is', 'a', 'little', 'girl', 'in', 'this', 'little', 'town']
['with', 'a', 'little', 'too', 'much', 'heart', 'to', 'go', 'around']
['live', 'forever', 'never', 'say', 'never']
['you', 'could', 'do', 'better', 'that', 'is', 'what', 'she', 'says']
['mama', 'named', 'her', 'lucky', 'on', 'a', 'star

['was', 'all', 'we', 'needed']
['you', 'drew', 'stars', 'around', 'my', 'scars']
['but', 'now', 'i', 'am', 'bleedin']
['because', 'i', 'knew', 'you']
['steppin', 'on', 'the', 'last', 'train']
['marked', 'me', 'like', 'a', 'bloodstain', 'i']
['i', 'knew', 'you']
['tried', 'to', 'change', 'the', 'ending']
['peter', 'losing', 'wendy', 'i']
['i', 'knew', 'you']
['leavin', 'like', 'a', 'father']
['running', 'like', 'water', 'i']
['because', 'when', 'you', 'are', 'young', 'they', 'assume', 'you', 'know', 'nothing']
['but', 'i', 'knew', 'you', 'would', 'linger', 'like', 'a', 'tattoo', 'kiss']
['i', 'knew', 'you', 'would', 'haunt', 'all', 'of', 'my', 'what-ifs']
['the', 'smell', 'of', 'smoke', 'would', 'hang', 'around', 'this', 'long']
['because', 'i', 'knew', 'everything', 'when', 'i', 'was', 'young']
['i', 'knew', 'i', 'would', 'curse', 'you', 'for', 'the', 'longest', 'time']
['chasin', 'shadows', 'in', 'the', 'grocery', 'line']
['i', 'knew', 'you', 'would', 'miss', 'me', 'once', 'the', 'thr

['disappear', 'when', 'you', 'come', 'back']
['everything', 'is', 'better']
['and', 'right', 'before', 'your', 'eyes']
['i', 'am', 'aching']
['no', 'fuss', 'nowhere', 'to', 'hide']
['just', 'you', 'and', 'me']
['e-he-e-he-e-he-e-he-e-he-e-he-e-he-e-he']
['this', 'is', 'the', 'last', 'time', 'i', 'am', 'asking', 'you', 'this']
['put', 'my', 'name', 'at', 'the', 'top', 'of', 'your', 'list']
['this', 'is', 'the', 'last', 'time', 'i', 'am', 'asking', 'you', 'why']
['you', 'break', 'my', 'heart', 'in', 'the', 'blink', 'of', 'an', 'eye']
['eye', 'eye']
['this', 'is', 'the', 'last', 'time', 'you', 'tell', 'me', 'i', 'have', 'got', 'it', 'wrong']
['this', 'is', 'the', 'last', 'time', 'i', 'say', 'it', 'is', 'been', 'you', 'all', 'along']
['this', 'is', 'the', 'last', 'time', 'i', 'let', 'you', 'in', 'my', 'door']
['this', 'is', 'the', 'last', 'time', 'i', 'will', 'not', 'hurt', 'you', 'anymore']
['oh-ho', 'oh-ho', 'oh-ho', 'oh-ho']
['oh-ho', 'oh-ho', 'oh-ho', 'oh-ho']
['this', 'is', 'the', 'la

['with', 'you', 'i', 'servе', 'with', 'you', 'i', 'fall', 'down', 'down']
['watch', 'you', 'breathe', 'in', 'watch', 'you', 'breathing', 'out', 'out']
['with', 'you', 'i', 'serve', 'with', 'you', 'i', 'fall', 'down', 'down']
['watch', 'you', 'breathe', 'in', 'watch', 'you', 'breathing', 'out', 'out']
['only', 'twenty', 'minutes', 'to', 'sleep']
['but', 'you', 'dream', 'of', 'some', 'epiphany']
['just', 'one', 'single', 'glimpse', 'of', 'relief']
['to', 'make', 'some', 'sense', 'of', 'what', 'you', 'have', 'seen']
['make', 'sense', 'of', 'what', 'you', 'have', 'seen']
['we', 'could', 'leave', 'the', 'christmas', 'lights', 'up', "'til", 'january']
['this', 'is', 'our', 'place', 'we', 'make', 'the', 'rules']
['and', 'there', 'is', 'a', 'dazzling', 'haze', 'a', 'mysterious', 'way', 'about', 'you', 'dear']
['have', 'i', 'known', 'you', '20', 'seconds', 'or', '20', 'years']
['can', 'i', 'go', 'where', 'you', 'go']
['can', 'we', 'always', 'be', 'this', 'close']
['forever', 'and', 'ever', 'ah'

['i', 'wish', 'i', 'could', 'go', 'back', 'to', 'my', 'old', 'self']
['not', 'knowing', 'you']
['my', 'old', 'self', 'would', 'never', 'get', 'into', 'this', 'much', 'love']
['it', 'is', 'strange', 'to', 'think', 'the', 'songs', 'we', 'used', 'to', 'sing']
['the', 'smiles', 'the', 'flowers', 'everything', 'is', 'gone']
['yesterday', 'i', 'found', 'out', 'about', 'you']
['even', 'now', 'just', 'lookin', 'at', 'you', 'feels', 'wrong']
['you', 'say', 'that', 'you', 'would', 'take', 'it', 'all', 'back']
['given', 'one', 'chance']
['it', 'was', 'a', 'moment', 'of', 'weakness']
['and', 'you', 'said', 'yes']
['you', 'should', 'have', 'said', 'no', 'you', 'should', 'have', 'gone', 'home']
['you', 'should', 'have', 'thought', 'twice', "'fore", 'you', 'let', 'it', 'all', 'go']
['you', 'should', 'have', 'known', 'that', 'word']
["'bout", 'what', 'you', 'did', 'with', 'her', 'd', 'get', 'back', 'to', 'me', 'get', 'back', 'to', 'me']
['and', 'i', 'should', 'have', 'been', 'there', 'in', 'the', 'bac

In [15]:
tokenizer = Tokenizer()
flatten_genre = []
for song in all_songs:
    for line in song:
        flatten_genre.append(line)

tokenizer.fit_on_texts(flatten_genre)
word_embeddings = tokenizer.texts_to_sequences(flatten_genre)
# print(flatten_genre[0:10])
# print(word_embeddings[0:10])
print('num words: ', len(word_embeddings))
# to_categorical needs this idk why
vocab_size = len(tokenizer.word_index) + 1
print('vocab size: ', vocab_size)

num words:  22643
vocab size:  4886


In [16]:
def generate_ngram_training_samples(word_embeddings, n) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    word embeddigns = list
    n = size of n gram
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    full_list = []
    for we in word_embeddings:
        full_list.extend(we)
    ngram_list_tup = ngrams(full_list, n)
    ngram_list = []
    for tup in ngram_list_tup:
        as_list = list(tup)
        ngram_list.append(as_list)
    return ngram_list

In [20]:
ngrams_list = generate_ngram_training_samples(word_embeddings, NGRAM_SIZE)
print(ngrams_list[:10])
print('num ngrams: ', len(ngrams_list))
# print(ngrams_list)

[[1, 355, 107], [355, 107, 3], [107, 3, 184], [3, 184, 43], [184, 43, 2], [43, 2, 3], [2, 3, 458], [3, 458, 24], [458, 24, 335], [24, 335, 21]]
num ngrams:  170055


In [23]:
def get_X_y(ngrams_list):
    '''
    Splits n-grams into X predictor matrix and associated y label vector
    Parameters:
    ngrams_list = list of ngrams in form of [[0, 1, 2], ....]
    return: 
    tuple (X, y)
    '''
    X = []
    y = []
    # splits ngrams into [a, b], [c]
    for ng in ngrams_list:
        x = []
        for i in range(0, len(ng)):
            if i < len(ng) - 1:
                #predictor
                x.append(ng[i])
            else:
                #label
                y.append(ng[i])
        X.append(x)
    # note both need to be np.arrays to work with model
    return (np.array(X), np.array(y))

In [24]:
X_pred, y_label = get_X_y(ngrams_list)

print(X_pred[0])
print(y_label[0])
print('X shape: ', X_pred.shape)
print('y shape: ', y_label.shape)

# turn labels into categories using keras utils
y_label = ku.to_categorical(y_label, num_classes=vocab_size)

[  1 355]
107
X shape:  (170055, 2)
y shape:  (170055,)


In [32]:
def create_model(pred, label, vocab_size, max_seq_len, epochs):
    '''
    Creates a model using LSTM
    '''
    model = Sequential()
    # todo: maybe needs input_len
    # todo: decide if output dim needs to be a diff size 
    model.add(Embedding(vocab_size, 32))
    model.add(LSTM(150, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(vocab_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
             optimizer='sgd',
             metrics=['accuracy'])
    
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    
    model.fit(pred, label, epochs=epochs, callbacks=[earlystop])
    print(model.summary())
    
    return model

In [35]:
lyric_model = create_model(X_pred, y_label, vocab_size, 2, 25)

Epoch 1/25
5315/5315 [==============================] - 43s 7ms/step - loss: 8.1713 - accuracy: 0.0539
Epoch 2/25
5315/5315 [==============================] - 40s 8ms/step - loss: 7.2535 - accuracy: 0.0539
Epoch 3/25
5315/5315 [==============================] - 39s 7ms/step - loss: 6.3505 - accuracy: 0.0539
Epoch 4/25
5315/5315 [==============================] - 40s 8ms/step - loss: 6.0727 - accuracy: 0.0533
Epoch 5/25
5315/5315 [==============================] - 40s 7ms/step - loss: 5.9764 - accuracy: 0.0543
Epoch 6/25
5315/5315 [==============================] - 40s 8ms/step - loss: 5.9304 - accuracy: 0.0541
Epoch 7/25
5315/5315 [==============================] - 40s 8ms/step - loss: 5.9041 - accuracy: 0.0540
Epoch 8/25
5315/5315 [==============================] - 40s 8ms/step - loss: 5.8876 - accuracy: 0.0534
Epoch 9/25
5315/5315 [==============================] - 38s 7ms/step - loss: 5.8773 - accuracy: 0.0535
Epoch 10/25
5315/5315 [==============================] - 38s 7ms/step - l

                                                                 
 lstm_10 (LSTM)              (None, None, 150)         109800    
                                                                 
 dropout_10 (Dropout)        (None, None, 150)         0         
                                                                 
 lstm_11 (LSTM)              (None, 100)               100400    
                                                                 
 dropout_11 (Dropout)        (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 4886)              493486    
                                                                 
Total params: 860,038
Trainable params: 860,038
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
def generate_text(tokenizer, model, seed_text, next_words, max_sequence_len):
#     lines = []
    LINE_LEN = np.random.randint(1, max_sequence_len)
    words = []
    words.extend(seed_text.split())
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
#         print('-----')
#         print(predicted.shape)
#         choice = np.argmax(predicted)
        choice = np.random.choice(len(predicted[0]), p=predicted[0])
#         choice = sample(predicted[0])
#         print('choice: ', choice)
#         print(index_of)
        word = tokenizer.index_word[choice]
        words.append(word)
#         print('word: ', word)
        if (len(words) % LINE_LEN == 0): # new line time
#             print(words)
#             print(len(words))
            words.append('\n')
            seed_text = word
        else:
            seed_text = seed_text + " " + word
#         seed_text = word
#         output_word = ""
#         for word, index in tokenizer.word_index.items():
#             if index == predicted[0]:
#                 output_word = word
#                 break
#         seed_text += " " + output_word
    return format_lyrics(words)

In [43]:
def sample(preds, temperature=1):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [44]:
def format_lyrics(words):
    lyr = ""
    for word in words:
        lyr = lyr + " " + word
    return lyr

In [45]:
max_line_len = 10
generated = generate_text(tokenizer, lyric_model, "you", 50, max_line_len)
print(generated)

 you making back say of 
 going 's know he 
 oh-oh sleep one said 
 the are i you 
 on i you 's 
 i was said go 
 am and to you 
 a is back am 
 is do should not 
 we up you the 
 when not to the 
 can tell a an 
 of who


In [46]:
generated2 = generate_text(tokenizer, lyric_model, "i", 50, max_line_len)
print(generated2)

 i step you can pulled can 
 to car even me you 
 your the never not it 
 the the because in me 
 but my it if of 
 with in i all it 
 this ooh stay would in 
 same are long my so 
 do just hide of are 
 i because bed now right 



In [47]:
generated3 = generate_text(tokenizer, lyric_model, "this is", 50, max_line_len)
print(generated3)

 this is what my pick him 
 red hello the of like 
 at in in fly you 
 we and me and am 
 else your the let not 
 have know like the time 
 are just was oh-oh-oh not 
 another your a of the 
 to and the 's i 
 but you that now how 
 it


In [48]:
generated4 = generate_text(tokenizer, lyric_model, "his", 50, max_line_len)
print(generated4)

 his one you could she have 
 could welcome he over you 
 wrong knows it he and 
 i you could to to 
 where me you so do 
 them he minds in do 
 the in thought dance smile 
 wrong me you right up 
 because you because she could 
 the my know and your 



In [49]:
generated5 = generate_text(tokenizer, lyric_model, "my", 50, max_line_len)
print(generated5)

 my and me were oh 
 them hands your home 
 are the the a 
 but but dressed he 
 these going a all 
 do my is tell 
 me like you better 
 made it a you 
 about have will and 
 my in in do 
 i going room in 
 i we ruining and 
 him and
